# imports

In [3]:
import sys
sys.path.insert(1,'../../code')
import myFunctions as mf
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import pandas as pd

# sit to stand function

In [4]:
def findSitToStand(tnorm_pelz,norm_pelz,buf_len=50,silence_thresh=0.01,peak_height=0.3,show=True,fig_num=0):
    #looking from when the silence ends untill the next big enough pick
    norm_pelzd=np.abs(np.diff(norm_pelz)/max(np.diff(norm_pelz)))
    norm_pelzd=np.append(norm_pelzd,norm_pelzd[-1])
    
    peaks,_=scipy.signal.find_peaks(norm_pelz,height=peak_height)
#     print('peaks')
    if len(peaks)==0:
        return [-1,-1]
    for index in range(peaks[0]-180,peaks[0]-buf_len):
        buf=norm_pelzd[index:index+buf_len]
        buf[buf<silence_thresh]=0
        if np.count_nonzero(buf)==buf_len:
            break
   
    if show==True:
        window=150
        plt.figure(fig_num,figsize=(8,8))
        plt.plot(tnorm_pelz,norm_pelz)
#         print('borm_pelz.Index[0] ',norm_pelz.index[0])
#         print('borm_pelz.Index ',norm_pelz.index)
#         print('peaks[0]+norm_pelz.index[0] ',peaks[0]+norm_pelz.index[0])
#         print('TYPE(norm_pelz) ',type(norm_pelz))
        plt.plot(tnorm_pelz[peaks[0]+norm_pelz.index[0]],norm_pelz[peaks[0]+norm_pelz.index[0]],'x')
        plt.plot(tnorm_pelz,norm_pelzd)
        plt.plot(tnorm_pelz[norm_pelzd<0.01],norm_pelzd[norm_pelzd<0.01],'*')
        plt.xlim([tnorm_pelz[max(norm_pelz.index[0],peaks[0]+norm_pelz.index[0]-window)],tnorm_pelz[peaks[0]+norm_pelz.index[0]+window]])
#         print(index)
#         print(peaks[0])
        plt.xticks([data_time[i] for i in range(index+norm_pelz.index[0],peaks[0]+norm_pelz.index[0],int(np.floor((peaks[0]-index)/10)))],\
                   [[data_time[i].hour,data_time[i].minute,data_time[i].second,data_time[i].microsecond]\
                    for i in range(index+norm_pelz.index[0],peaks[0]+norm_pelz.index[0],int(np.floor((peaks[0]-index)/10)))],rotation=90)
        plt.plot(tnorm_pelz[index+norm_pelz.index[0]],norm_pelz[index+norm_pelz.index[0]],'o')
        plt.legend(['pelz','peak','diff_pelz','diff_pelz<'+str(silence_thresh),'end of silence'])
        plt.grid(True)
        plt.show()
#         print('')
    return [index+norm_pelz.index[0],peaks[0]+norm_pelz.index[0]]

start=0
end=0
# getUp=findSitToStand(data_time,df2['Pelvis z'],buf_len=30,silence_thresh=0.01,peak_height=0.8,show=True)
getUpAll=[]

def findSitToStandAll(tnorm_pelz,norm_pelz,buf_len=30,silence_thresh=0.01,peak_height=0.8,show=True):
    start=0
    end=0
    ind=0
    getUpAll=[]
    while end!=-1:
        getUp=findSitToStand(pd.Series(tnorm_pelz[start:],index=range(start,len(tnorm_pelz))),\
                             pd.Series(norm_pelz[start:],index=range(start,len(norm_pelz))),\
                             buf_len=buf_len,silence_thresh=silence_thresh,peak_height=peak_height,show=show,fig_num=ind)
        if show==True:    
            print(getUp)
        if -1 in getUp:
            return getUpAll
        getUpAll.append(getUp)
        # the new start will be the index>gu[1] which np.abs(norm_pelz[index]-norm_pelz[gu[0]]
        start=[i for i in range(getUp[1],len(tnorm_pelz)) if np.abs(norm_pelz[i]-norm_pelz[getUp[0]]) < 0.01]
        if len(start)==0:
             return getUpAll
        start=start[0]
        end=getUp[1]
        ind=ind+1
    return getUpAll
# getUp=findSitToStandAll(data_time,df2['Pelvis z'],buf_len=50,silence_thresh=0.01,peak_height=0.8,show=True)
# print(getUp)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import LeavePGroupsOut
from collections import Counter
import random

def generateLSTMData(path,files,fileName):
    cut=[]
    array=[[]]
    arrayP=[[]]
    labeldOut=[[]]
    data=[]
    for i,file in enumerate(files):
        #set mesurements
        t = time.time()
        df1,df2,df3,data_time=loadData(path,file)
        elapsed = time.time() - t
        print('elapsed all loadData ',elapsed)
        #cut mesurement df1 and df2
        arrayOut=df1
        arrayOutP=df2
        #find getUp moments
        getUp=findSitToStandAll(data_time,df2['Pelvis z'],buf_len=50,silence_thresh=0.01,peak_height=0.8,show=False)        
        print('name ',file,' len(getUp)',len(getUp))
        #create array with labels
        labeld=checkForGetUp(arrayOut.iloc[:,1],getUp)
        # over sample
        print('len array', len(arrayOut))
        print('1s train', np.count_nonzero(labeld==1.0))
        arrayOut,labeld,arrayOutP=overSampleData(arrayOut,labeld,arrayOutP,getUp)
        #add id to arrays
        arrayOut=addID(arrayOut,i)
        arrayOutP=addID(arrayOutP,i)
        #add ind to arrays
        arrayOut=addIND(arrayOut,i)
        arrayOutP=addIND(arrayOutP,i)
        #accumelate arrays
        array,labeldOut=joinArrays(array,arrayOut),joinArrays(labeldOut,labeld)
        arrayP=joinArrays(arrayP,arrayOutP) 
    
    #divide into test\train
    Xtrain,ytrain,Xtest,ytest,Xtestp=divideTestTrain(X=array,y=labeldOut,precent1=0.4,Xp=arrayP,num=1)
    Xtest,ytest,Xvalid,yvalid,Xtestp2=divideTestTrain(X=Xtest,y=ytest,precent1=0.5,Xp=Xtestp,num=2)
    
#     write txt files
    print(np.shape(Xtest))
    writeTxt2(path,Xtest[:,:3],'regular','test','x'),writeTxt2(path,ytest,'labeld','test','x')
    writeTxt2(path,Xtrain[:,:3],'regular','train','x'),writeTxt2(path,ytrain,'labeld','train','x')
    writeTxt2(path,Xvalid[:,:3],'regular','valid','x'),writeTxt2(path,yvalid,'labeld','valid','x')
    writeTxt2(path,Xtestp2[:,:3],'regular','testp','x')
#     writeTxt2(path,trainzP,'Pelvis','train','z'),writeTxt2(path,testzP,'Pelvis','test','z')

    print('I wrote')
    #count getUps and count total
    print('len arra', len(array))
    print('len arrayP', len(arrayP))
    print('1s train', np.count_nonzero(ytrain==1.0))
    print('len train y', len(ytrain))
    print('len train X', len(Xtrain))
    print('1s test', np.count_nonzero(ytest==1.0))
    print('len test y', len(ytest))
    print('len test X', len(Xtest))
    print('1s valid', np.count_nonzero(yvalid==1.0))
    print('len valid y', len(yvalid))
    print('len valid X', len(Xvalid))
    print('len testp X', len(Xtestp2))
    return arrayx,labeldOut

def partition_on_index(it, indices):
    indices = set(indices)   # convert to set for fast lookups
    l1, l2 = [], []
    l_append = (l1.append, l2.append)
    for idx, element in enumerate(it):
        l_append[idx in indices](element)
    return l1, l2

def divideTestTrain(X,y,precent1=0.2,Xp=None,num=1):
    # initiate 
    train_ind,test_ind=[],[]
    #count groups
    num_groups=int(max(X[:,3])+1)
    print('num_groups ',num_groups)
    #divide to number of patient in each group
    train_num=int(np.floor(num_groups*(1-precent1)))
    print('train_num ',train_num)
    #shuffle
    yy = random.sample(range(num_groups), train_num)
    print('yy ',yy)
    #find indeces
    for val in yy:
        indd=np.where(X[:,3]==val)
        for item in indd:
            for it in item:
                train_ind.append(it)
    #assign arrays
    Xtest,Xtrain=partition_on_index(X, train_ind)
    if num==1:
        Xtestp,_=partition_on_index(Xp, train_ind)
    else:
        _,Xtestp=partition_on_index(Xp, train_ind)
    ytest,ytrain=partition_on_index(y, train_ind)
    print('len Xtrain ',len(Xtrain))
    print('len ytrain ',len(ytrain))
    print('len Xtest ',len(Xtest))
    print('len ytest ',len(ytest))
    return np.asarray(Xtrain),np.asarray(ytrain),np.asarray(Xtest),np.asarray(ytest),np.asarray(Xtestp)

def overSampleData(X,y,Xp,getUp):
    addedX=np.empty((0,np.shape(X)[1]))
    addedXp=np.empty((0,np.shape(Xp)[1]))
    addedy=np.empty((0,))
    newy=y
    newX=X
    newXp=Xp
#     print('len(y) ',len(y))
#     print('newy ',newy)
#     print('y.dtype ',y.dtype)
    #create the added
    for gu in getUp:
        addedy=np.concatenate((addedy,y[gu[0]-15:gu[1]+15]), axis=None)
        addedX=np.concatenate((addedX,X[gu[0]-15:gu[1]+15]), axis=0)
        addedXp=np.concatenate((addedXp,Xp[gu[0]-15:gu[1]+15]), axis=0)
#     print('np.count_nonzero(addedy==1.0) ',np.count_nonzero(addedy==1.0))
#     print('np.count_nonzero(addedy==0.0) ',np.count_nonzero(addedy==0.0))
#     print('len(addedy) ',len(addedy))
#     print('addedy ',addedy)
    #add the added signal the needed number of times
    while np.count_nonzero(newy==1.0)<np.count_nonzero(newy==0.0):
#         print('np.count_nonzero(newy==1.0) ',np.count_nonzero(newy==1.0))
#         print('np.count_nonzero(newy==0.0) ',np.count_nonzero(newy==0.0))
#         print()
        newy=np.concatenate((newy,addedy), axis=None)
        newX=np.concatenate((newX,addedX), axis=0)
        newXp=np.concatenate((newXp,addedXp), axis=0)
#     print('len X ',len(X))
#     print('len y ',len(y))
#     print('np.count_nonzero(~np.isnan(y))', np.count_nonzero(~np.isnan(y)))
#     print('len newX ',len(newX))
#     print('len newy ',len(newy))
#     print('np.count_nonzero(~np.isnan(newy))', np.count_nonzero(~np.isnan(newy)))
    return newX, newy, newXp
    
def vizualise(zp,label):
    print('shape zp ',np.shape(zp))
    ind=np.where(label==1.0)
#     print('shape ind ',np.shape(ind))
#     print('shape label ',np.shape(label))
#     print('ind[0,:] ',ind[0])
#     print('ind ',ind)
#     print('label ',label)
    for i in ind[0]:
        print('i ',i)
        print('shape zp[i] ',np.shape(zp[i]))
        plt.figure()
        plt.ylim(0.4,1)
        plt.plot(zp[i,:])
        
        plt.show()
        
def cutMes(df1,length=64):
    cuttedx=np.reshape(df1.iloc[:int(len(df1.iloc[:,0])/length)*length,0].to_numpy()\
                           ,(int(len(df1.iloc[:,0])/length),length))
    cuttedy=np.reshape(df1.iloc[:int(len(df1.iloc[:,1])/length)*length,1].to_numpy()\
                           ,(int(len(df1.iloc[:,1])/length),length))
    cuttedz=np.reshape(df1.iloc[:int(len(df1.iloc[:,2])/length)*length,2].to_numpy()\
                           ,(int(len(df1.iloc[:,2])/length),length))
    return cuttedx,cuttedy,cuttedz

def checkForGetUp(arrayOut,getUp):
    labelOut=np.zeros((np.shape(arrayOut)[0],1))
    for ind,_ in enumerate(arrayOut):
        for gu in getUp:
            #all the getUp is in segment
            if ind>gu[0] and ind<gu[1]:
                labelOut[ind]=1.0
    return labelOut

def overlapData(cutted):
    arrayOut=np.zeros((len(cutted),64*2))
    for i in range(0,np.shape(cutted)[0]-1):
        arrayFill=np.concatenate((cutted[i],cutted[i+1]),axis=0)
        arrayOut[i]=(arrayFill)
    return arrayOut

def joinArrays(array,cuted):
    if min(np.shape(array))==0:
        array=cuted
    else:
        array=np.concatenate((array,cuted),axis=0)
    return array

def loadData(path,file):
    df1=mf.readXsens(path+file+'/'+file+'.csv',cols=['Right Forearm x','Right Forearm y','Right Forearm z'],sheet='Segment Position')
    df2=mf.readXsens(path+file+'/'+file+'.csv',cols=['Pelvis x','Pelvis y','Pelvis z'],sheet='Segment Position')
    df3=mf.readXsens(path+file+'/'+file+'.csv',cols=['Right Forearm x','Right Forearm y','Right Forearm z'],sheet='Segment Orientation - Euler')
#     data_time=mf.getFrameXsens(path+file+'/'+file+'.mvnx')
#     data_time=mf.parseXsensTime(data_time,len(df1['Right Forearm x']),60)
    return df1,df2,df3,list(range(0,len(df1)))

def writeTxt2(path,array,type1,type2,axis):
    if not os.path.exists(path+'myne'):
        os.mkdir(path+'myne')
    if type1=='labeld':        
        fileObj=open(path+'myne/'+'/y_'+type2+'.txt','w')
        for line in array:
            fileObj.write(str(line))
            fileObj.write('\n')
        fileObj.close()
        return
    elif type1=='Pelvis':        
        fileObj=open(path+'myne/'+'/z_'+type2+'P.txt','w')  
    else:
        fileObj=open(path+'myne/'+'/'+'X'+type2+'.txt','w')                            
    for line in array:
        fileObj.writelines(["%s " % item  for item in line])
        fileObj.write('\n')
    fileObj.close()

def shuffleTogether(array1,array2,array3,array4):
    indices = np.arange(array1.shape[0])
    np.random.shuffle(indices)
    array1 = array1[indices]
    array2 = array2[indices]
    return array1,array2,array3,array4,indices

def addID(array,id):
    arrayOut=np.ones((array.shape[0],array.shape[1]+1))*id
    arrayOut[:,:-1]=array
    return arrayOut

def addIND(array,id):
    arrayOut=np.ones((array.shape[0],array.shape[1]+1))
    arrayOut[:,-1]=range(len(array))
    arrayOut[:,:-1]=array
    return arrayOut
    
# def visualizeData()
import os
import random
import time
import shutil
path='C:/Users/User/Desktop/yocahi/master/getUp/data/pilot/'
files = list(os.listdir(path))
files = list(next(os.walk(path))[1])
if os.path.exists(path+'myne'):
    shutil.rmtree(path+'myne')
# print(files)
# print(list(enumerate(files)))
# xxx=list(enumerate(files))
# random.shuffle(xxx)
# print(xxx)
# %prun generateLSTMData(path,files,path+'out.txt')
data=generateLSTMData(path,files,path+'out.txt')

exist
exist
exist
elapsed all loadData  5.22135329246521
name  arik  len(getUp) 5
len array 36830
1s train 428
exist


KeyboardInterrupt: 

In [7]:
59999/443939
1-384/59999

0.9935998933315555